In [8]:
import pandas as pd

file_path = '..data/curated/property/property 2.csv'
df = pd.read_csv(file_path)

# calculate size of each
batch_size = len(df) // 5

# split into 5 
df_parta = df.iloc[:batch_size]
df_partb = df.iloc[batch_size:batch_size*2]
df_partc = df.iloc[batch_size*2:batch_size*3]
df_partd = df.iloc[batch_size*3:batch_size*4]
df_parte = df.iloc[batch_size*4:]

# save each part as a new file
output_path_parta = '../data/raw/external/API/2/property_addresses_part2a.csv'
output_path_partb = '../data/raw/external/API/2/property_addresses_part2b.csv'
output_path_partc = '../data/raw/external/API/2/property_addresses_part2c.csv'
output_path_partd = '../data/raw/external/API/2/property_addresses_part2d.csv'
output_path_parte = '../data/raw/external/API/2/property_addresses_part2e.csv'

df_parta.to_csv(output_path_parta, index=False)
df_partb.to_csv(output_path_partb, index=False)
df_partc.to_csv(output_path_partc, index=False)
df_partd.to_csv(output_path_partd, index=False)
df_parte.to_csv(output_path_parte, index=False)

print(f"file split into: {output_path_parta}, {output_path_partb}, {output_path_partc}, {output_path_partd}, 和 {output_path_parte}")


文件已分割并保存为: /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parta.csv, /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partb.csv, /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partc.csv, /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_partd.csv, 和 /Users/fenglingyi/Downloads/groupwork_generate/property_addresses_parte.csv


<p style="font-size:24px;">A</p>


In [11]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection


houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part2a.csv')
schools_df = pd.read_csv('../data/raw/external/API/victoria_schools.csv', encoding='ISO-8859-1') 

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'), 
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


houses_geojson_output_path = '../data/raw/external/API/2/property_geojson_2a.geojson'
schools_geojson_output_path = '../data/raw/external/API/2/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("Created")


房屋和车站的GeoJSON文件已生成


In [14]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf624880467d82d87348ab87a207c27ad5516c'
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/2/property_geojson_2a.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):

                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)
    
    return all_shortest_distances

shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/2/a_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"save to {output_path}")


结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/a_closest_school_distance_final_2.csv


<p style="font-size:24px;">B</p>


In [1]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part2b.csv') 
schools_df = pd.read_csv('../data/raw/external/victoria_schools.csv', encoding='ISO-8859-1')

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'), 
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)

houses_geojson_output_path = '../data/raw/external/API/2/property_geojson_2b.geojson'
schools_geojson_output_path = '../data/raw/external/API/2/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("Created")


房屋和车站的GeoJSON文件已生成


In [2]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf624880467d82d87348ab87a207c27ad5516c'
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/2/property_geojson_2b.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1) 
    
    return all_shortest_distances

shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/2/b_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"save to {output_path}")

结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/b_closest_school_distance_final_2.csv


<p style="font-size:24px;">C</p>


In [3]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part2c.csv') 
schools_df = pd.read_csv('../data/raw/external/victoria_schools.csv', encoding='ISO-8859-1')

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'), 
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)


schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


houses_geojson_output_path = '../data/raw/external/API/2/property_geojson_2c.geojson'
schools_geojson_output_path = '../data/raw/external/API/2/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("Created")

房屋和车站的GeoJSON文件已生成


In [6]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf6248e145f61cd48746779f79c90e911dfd2b'
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/2/property_geojson_2c.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
         
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)
    
    return all_shortest_distances

shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/2/c_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved to {output_path}")

Error calculating distance matrix: 500 ({'error': 'There was a problem proxying the request'})
结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/c_closest_school_distance_final_2.csv


<p style="font-size:24px;">D</p>


In [4]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection


houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part2d.csv') 
schools_df = pd.read_csv('../data/raw/external/victoria_schools.csv', encoding='ISO-8859-1') 

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'), 
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


houses_geojson_output_path = '../data/raw/external/API/2/property_geojson_2d.geojson'
schools_geojson_output_path = '../data/raw/external/API/2/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("Created")

房屋和车站的GeoJSON文件已生成


In [7]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf62487af3a2624ef84344babc38f1f6fde19c'
client = openrouteservice.Client(key=api_key)


with open('../data/raw/external/API/2/property_geojson_2d.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)


house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)
    
    return all_shortest_distances

shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/2/d_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved to {output_path}")

结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/d_closest_school_distance_final_2.csv


<p style="font-size:24px;">E</p>


In [5]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/raw/external/API/2/property_addresses_part2e.csv')  
schools_df = pd.read_csv('../data/raw/external/victoria_schools.csv', encoding='ISO-8859-1') 

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
schools_df = schools_df.dropna(subset=['longitude', 'latitude'])


def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

schools_features = schools_df.apply(lambda row: create_geojson_feature(row, 'schools'), axis=1).tolist()
schools_feature_collection = FeatureCollection(schools_features)


houses_geojson_output_path = '../data/raw/external/API/2/property_geojson_2e.geojson'
schools_geojson_output_path = '../data/raw/external/API/2/vic_school_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(schools_geojson_output_path, 'w') as f:
    geojson.dump(schools_feature_collection, f)

print("created")

房屋和车站的GeoJSON文件已生成


In [8]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf62485f72fc1cea314a03ad514d2a07be9533'
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/2/property_geojson_2e.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/2/vic_school_geojson.geojson', 'r') as f:
    schools_geojson = geojson.load(f)

def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
schools_coordinates = extract_coordinates(schools_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
schools_coords = [(lon, lat) for lat, lon, _ in schools_coordinates]

def calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + schools_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(schools_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_schools_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_schools = schools_coordinates[closest_schools_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_schools': closest_schools,
                        'minimum_distance_school': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1) 
    
    return all_shortest_distances

shortest_distances = calculate_distance_matrix_in_batches(house_coords, schools_coords, batch_size=1)

shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/4/e_closest_school_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved to {output_path}")

结果已保存到 /Users/fenglingyi/Downloads/groupwork_generate/e_closest_school_distance_final_2.csv


In [9]:
import pandas as pd

file1 = '../data/raw/external/API/4/a_closest_school_distance_final_2.csv'
file2 = '../data/raw/external/API/4/b_closest_school_distance_final_2.csv'
file3 = '../data/raw/external/API/4/c_closest_school_distance_final_2.csv'
file4 = '../data/raw/external/API/4/d_closest_school_distance_final_2.csv'
file5 = '../data/raw/external/API/4/e_closest_school_distance_final_2.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)
df5 = pd.read_csv(file5)

# Merge all data boxes
merged_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# Save the merged data frame as a new CSV file
output_file = '../data/curated/external/API/2/closest_school_distance_final_2.csv'
merged_df.to_csv(output_file, index=False)

print(f"Results have been saved to {output_file}")


所有文件已合并并保存为: /Users/fenglingyi/Downloads/groupwork_final/closest_school_distance_final_2.csv
